# Basic EPR Data Loading and Visualization
## Using eprload and eprplot functions

This notebook demonstrates how to load and visualize EPR data using the core functions `eprload` and `eprplot`. We will work with real experimental data to show practical examples.

### What you will learn:
- How to use `eprload()` to load EPR data
- Understanding the data structure returned by eprload
- Using `eprplot` functions for 1D and 2D visualizations
- Working with different EPR data types (CW, time-domain, angular studies)

In [ ]:
import epyr
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Setup
data_dir = Path("../data")
plt.rcParams['figure.figsize'] = (10, 6)

print(f"EPyR Tools version: {epyr.__version__}")

## 1. Basic eprload usage

The `eprload()` function is the main function to load EPR data. It returns 4 values:
- `x`: axis data (field, time, etc.)
- `y`: signal data
- `params`: dictionary with all measurement parameters
- `filepath`: path to the loaded file

In [ ]:
# Load CW EPR data (field sweep)
cw_file = data_dir / "130406SB_CaWO4_Er_CW_5K_20.DSC"

# Load the data
x_field, y_signal, params_cw, filepath = epyr.eprload(str(cw_file), plot_if_possible=False)

print(f"Loaded file: {Path(filepath).name}")
print(f"Field axis shape: {x_field.shape}")
print(f"Signal shape: {y_signal.shape}")
print(f"Field range: {x_field.min():.1f} to {x_field.max():.1f} G")
print(f"Number of parameters: {len(params_cw)}")

## 2. 1D plotting with eprplot.plot_1d()

Use `eprplot.plot_1d()` to create professional EPR plots:

In [ ]:
# Plot 1D EPR spectrum
fig, ax = epyr.eprplot.plot_1d(x_field, y_signal, params_cw, 
                               title="Erbium in CaWO4 at 5K - CW EPR")
plt.show()

## 3. Time-domain EPR data

Now let's load time-domain data (Rabi oscillations):

In [ ]:
# Load time-domain EPR data (Rabi oscillations)
rabi_file = data_dir / "2024_08_CaWO4171Yb_rabi_6K_6724G_18dB.DSC"

x_time, y_rabi, params_rabi, filepath_rabi = epyr.eprload(str(rabi_file), plot_if_possible=False)

print(f"Loaded file: {Path(filepath_rabi).name}")
print(f"Time axis shape: {x_time.shape}")
print(f"Signal shape: {y_rabi.shape}")
print(f"Time range: {x_time.min():.1f} to {x_time.max():.1f} ns")
print(f"Signal is complex: {np.iscomplexobj(y_rabi)}")

In [ ]:
# Plot time-domain data
fig, ax = epyr.eprplot.plot_1d(x_time, y_rabi, params_rabi, 
                               title="Rabi Oscillations - Yb in CaWO4 at 6K")
plt.show()

## 4. 2D EPR Data - Angular Study

Load 2D data from an angular study of MgO:

In [ ]:
# Load 2D angular EPR data
angular_file = data_dir / "2014_03_19_MgO_300K_111_fullrotation33dB.par"

x_angular, y_angular, params_angular, filepath_angular = epyr.eprload(str(angular_file), plot_if_possible=False)

print(f"Loaded file: {Path(filepath_angular).name}")
print(f"Data shape: {y_angular.shape}")
print(f"X-axis type: {type(x_angular)}")

if isinstance(x_angular, list):
    print(f"Field axis: {len(x_angular[0])} points, {x_angular[0].min():.1f} to {x_angular[0].max():.1f} G")
    print(f"Angle axis: {len(x_angular[1])} points, {x_angular[1].min():.1f} to {x_angular[1].max():.1f} degrees")

### 4.1 2D Color Map

In [ ]:
# Plot 2D data as color map
fig, ax = epyr.eprplot.plot_2d_map(x_angular, y_angular, params_angular,
                                   title="MgO Angular Study - Color Map",
                                   cmap="RdBu_r")
plt.show()

### 4.2 2D Waterfall Plot

In [ ]:
# Plot 2D data as waterfall
fig, ax = epyr.eprplot.plot_2d_waterfall(x_angular, y_angular, params_angular,
                                          title="MgO Angular Study - Waterfall",
                                          max_traces=20, offset_factor=0.1)
plt.show()

### 4.3 Manual slices from 2D data

In [ ]:
# Extract and plot specific angle slices
if isinstance(x_angular, list):
    field_axis = x_angular[0]
    angle_axis = x_angular[1]
    
    # Select angles to plot
    angles_to_plot = [0, 45, 90, 135]
    
    plt.figure(figsize=(12, 8))
    
    for angle in angles_to_plot:
        # Find closest angle index
        angle_idx = np.argmin(np.abs(angle_axis - angle))
        actual_angle = angle_axis[angle_idx]
        
        # Plot slice
        plt.plot(field_axis, y_angular[angle_idx, :], 
                label=f'{actual_angle:.0f}°', linewidth=1.5)
    
    plt.xlabel('Magnetic Field (G)')
    plt.ylabel('EPR Intensity (a.u.)')
    plt.title('MgO Angular Study - Selected Slices')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 5. 2D Rabi Data

Load another 2D dataset - Rabi oscillations as function of field:

In [ ]:
# Load 2D Rabi data
rabi2d_file = data_dir / "Rabi2D_GdCaWO4_6dB_3770G_2.DSC"

x_rabi2d, y_rabi2d, params_rabi2d, filepath_rabi2d = epyr.eprload(str(rabi2d_file), plot_if_possible=False)

print(f"Loaded file: {Path(filepath_rabi2d).name}")
print(f"Data shape: {y_rabi2d.shape}")
print(f"Signal is complex: {np.iscomplexobj(y_rabi2d)}")

if isinstance(x_rabi2d, list):
    print(f"Axis 1: {len(x_rabi2d[0])} points, {x_rabi2d[0].min():.1f} to {x_rabi2d[0].max():.1f}")
    print(f"Axis 2: {len(x_rabi2d[1])} points, {x_rabi2d[1].min():.1f} to {x_rabi2d[1].max():.1f}")

### 5.1 2D Rabi Color Map

In [ ]:
# Plot 2D Rabi data as color map
fig, ax = epyr.eprplot.plot_2d_map(x_rabi2d, y_rabi2d, params_rabi2d,
                                   title="2D Rabi - Gd in CaWO4 - Color Map",
                                   cmap="viridis")
plt.show()

### 5.2 2D Rabi Waterfall

In [ ]:
# Plot 2D Rabi data as waterfall
fig, ax = epyr.eprplot.plot_2d_waterfall(x_rabi2d, y_rabi2d, params_rabi2d,
                                          title="2D Rabi - Gd in CaWO4 - Waterfall",
                                          max_traces=15)
plt.show()

### 5.3 Manual slices from 2D Rabi data

In [ ]:
# Plot selected slices from 2D Rabi data
if isinstance(x_rabi2d, list):
    axis1 = x_rabi2d[0]
    axis2 = x_rabi2d[1]
    
    # Select a few representative slices
    n_slices = 5
    slice_indices = np.linspace(0, len(axis2)-1, n_slices, dtype=int)
    
    plt.figure(figsize=(12, 8))
    
    # Use magnitude for complex data
    plot_data = np.abs(y_rabi2d) if np.iscomplexobj(y_rabi2d) else y_rabi2d
    
    for i, idx in enumerate(slice_indices):
        offset = i * np.max(plot_data) * 0.1  # Small vertical offset
        plt.plot(axis1, plot_data[idx, :] + offset, 
                label=f'Slice {idx} ({axis1[idx]:.1f})', linewidth=1.5)
    
    plt.xlabel('Time or Field axis')
    plt.ylabel('Signal Magnitude (offset)')
    plt.title('2D Rabi Data - Selected Slices')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 6. Examining EPR parameters

The params dictionary contains all measurement parameters:

In [ ]:
# Show key parameters for CW data
print("CW EPR Parameters (sample):")
print("-" * 30)

key_params = ['MWFQ', 'B0VL', 'BWVL', 'MWPW', 'AVGS', 'SPTP']
for param in key_params:
    if param in params_cw:
        print(f"{param}: {params_cw[param]}")

print(f"\nTotal parameters: {len(params_cw)}")

In [ ]:
# Show parameters for time-domain data
print("Rabi Parameters (sample):")
print("-" * 25)

for param in key_params:
    if param in params_rabi:
        print(f"{param}: {params_rabi[param]}")

print(f"\nTotal parameters: {len(params_rabi)}")

## Summary

This notebook demonstrated:

### eprload function:
- Returns (x, y, params, filepath)
- Works with different EPR data formats
- Handles 1D and 2D data automatically

### eprplot functions:
- `plot_1d()`: for 1D EPR spectra and time traces
- `plot_2d_map()`: color map visualization of 2D data
- `plot_2d_waterfall()`: waterfall plots of 2D data

### Data types shown:
- CW EPR (field vs signal)
- Time-domain EPR (time vs signal)
- 2D angular studies (field vs angle vs signal)
- 2D Rabi data

The eprload and eprplot functions provide a simple interface for loading and visualizing EPR data from Bruker spectrometers.